In [1]:
# Предметная область: книги 
# Книги разных жанров.

In [10]:
import pandas as pd
import sqlite3
import json
import csv

In [11]:
file_path = "bookdata.csv"
data_csv = pd.read_csv(file_path, sep=',')
data_csv[0:10]

,availability,category,description,num_reviews,price,price_excl_tax,price_incl_tax,product_type,stars,tax,title,upc,url
0,19,default,"Since her assault, Miss Annette Chetwynd has b...",0,13.99,13.99,13.99,books,2,0.0,"Since her assault, Miss Annette Chetwynd has b...",0312262ecafa5a40,https://books.toscrape.com/catalogue/starving-...
1,20,fiction,"Dans une France assez proche de la nôtre, un h...",0,50.10,50.10,50.10,books,1,0.0,"Dans une France assez proche de la nôtre, un h...",6957f44c3847a760,https://books.toscrape.com/catalogue/soumissio...
2,19,young adult,Patient Twenty-nine.A monster roams the halls ...,0,22.65,22.65,22.65,books,1,0.0,Patient Twenty-nine.A monster roams the halls ...,f77dbf2323deb740,https://books.toscrape.com/catalogue/the-requi...
3,20,history,From a renowned historian comes a groundbreaki...,0,54.23,54.23,54.23,books,5,0.0,From a renowned historian comes a groundbreaki...,4165285e1663650f,https://books.toscrape.com/catalogue/sapiens-a...
4,20,historical fiction,"""Erotic and absorbing...Written with starling ...",0,53.74,53.74,53.74,books,1,0.0,"""Erotic and absorbing...Written with starling ...",90fa61229261140a,https://books.toscrape.com/catalogue/tipping-t...
5,19,default,"""If you have a heart, if you have a soul, Kare...",0,17.93,17.93,17.93,books,3,0.0,"""If you have a heart, if you have a soul, Kare...",e72a5dfc7e9267b2,https://books.toscrape.com/catalogue/the-comin...
6,19,default,For readers of Laura Hillenbrand's Seabiscuit ...,0,22.60,22.60,22.60,books,4,0.0,For readers of Laura Hillenbrand's Seabiscuit ...,e10e1e165dc8be4a,https://books.toscrape.com/catalogue/the-boys-...
7,19,business,Drawing on his extensive experience evaluating...,0,33.34,33.34,33.34,books,4,0.0,Drawing on his extensive experience evaluating...,2597b5a345f45e1b,https://books.toscrape.com/catalogue/the-dirty...
8,19,poetry,"Praise for Aracelis Girmay:""[Girmay's] every l...",0,52.15,52.15,52.15,books,1,0.0,"Praise for Aracelis Girmay:""[Girmay's] every l...",1dfe412b8ac00530,https://books.toscrape.com/catalogue/the-black...
9,22,poetry,It's hard to imagine a world without A Light i...,0,51.77,51.77,51.77,books,3,0.0,It's hard to imagine a world without A Light i...,a897fe39b1053632,https://books.toscrape.com/catalogue/a-light-i...


In [12]:
def connect_to_db(file):
    connection = sqlite3.connect(file)
    connection.row_factory = sqlite3.Row
    return connection

In [13]:
def create_three_table():
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS books_1
               (id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT,
                availability INTEGER,
                category TEXT,
                num_reviews INTEGER)''')
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS books_2
               (id INTEGER PRIMARY KEY AUTOINCREMENT,
                description TEXT,
                product_type TEXT,
                price REAL,
                price_excl_tax REAL,
                price_incl_tax REAL)''')
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS books_3
               (id INTEGER PRIMARY KEY AUTOINCREMENT,
                description TEXT,
                url TEXT,
                upc TEXT,
                tax REAL,
                stars INTEGER)''')
    conn.commit()
    conn.close()

In [14]:
def insert_data_into_tables():
    conn = sqlite3.connect('books.db')
    c = conn.cursor()
    with open('bookdata.csv', 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=',')
        for item in csv_reader:
            c.execute("INSERT INTO books_1 (title, availability, category, num_reviews) VALUES (?, ?, ?, ?)",
                      (item['title'], item['availability'], item['category'], item['num_reviews']))
            
            c.execute("INSERT INTO books_2 (description, product_type, price, price_excl_tax, price_incl_tax) VALUES ((SELECT id FROM books_1 WHERE title = ?), ?, ?, ?, ?)",
                      (item['title'], item['product_type'], item['price'], item['price_excl_tax'], item['price_incl_tax']))
            
            c.execute("INSERT INTO books_3 (description, url, upc, tax, stars) VALUES ((SELECT id FROM books_1 WHERE title = ?), ?, ?, ?, ?)",
                      (item['title'], item['url'], item['upc'], item['tax'], item['stars']))
            
    with open('Books.json', 'r', encoding='utf-8') as json_file:
        json_data = json.load(json_file)
        for data in json_data:
            c.execute("INSERT INTO books_1 (title, availability, category, num_reviews) VALUES (?, ?, ?, ?)",
                      (data['title'], data['availability'], data['category'], data['num_reviews']))
            
            c.execute("INSERT INTO books_2 (description, price) VALUES ((SELECT id FROM books_1 WHERE title = ?), ?)",
                      (data['title'], data['price']))
            
    conn.commit()
    c.close()         

In [24]:
def execute_query(query):
    conn = connect_to_db('books.db')
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    conn.close()
    return result

# Запрос 1: Выборка данных с простым условием, сортировкой и ограничением количества результатов
query1 = "SELECT * FROM books_1 WHERE category = 'young adult' ORDER BY num_reviews DESC LIMIT 10"
result1 = execute_query(query1)

# Запрос 2: Подсчет объектов по условию
query2 = "SELECT category, COUNT(*) as count FROM books_1 GROUP BY category"
result2 = execute_query(query2)

# Запрос 3: Группировка данных и расчет средней цены
query3 = "SELECT description, AVG(price) as avg_price FROM books_2 GROUP BY description"
result3 = execute_query(query3)

# Запрос 4: Обновление данных
query4 = "UPDATE books_1 SET availability = 0 WHERE category = 'business'"
execute_query(query4)

# Запрос 5: Выборка данных из нескольких таблиц с использованием JOIN
query5 = "SELECT b1.title, b2.price FROM books_1 b1 JOIN books_2 b2 ON b1.id = b2.id WHERE b1.availability = 19"
result5 = execute_query(query5)

# Запрос 6: Агрегация данных с использованием функции SUM
query6 = "SELECT category, SUM(num_reviews) as total_reviews FROM books_1 GROUP BY category"
result6 = execute_query(query6)

# Запрос 7: Выборка данных с несколькими условиями
query7 = "SELECT * FROM books_1 WHERE category = 'historical fiction' AND availability = 20"
result7 = execute_query(query7)

# Преобразование результатов в формат JSON
results = {
    'запрос1': [dict(row) for row in result1],
    'запрос2': [dict(row) for row in result2],
    'запрос3': [dict(row) for row in result3],
    'запрос5': [dict(row) for row in result5],
    'запрос6': [dict(row) for row in result6],
    'запрос7': [dict(row) for row in result7]
}

# Сохранение результатов в файле JSON
with open('результаты_запросов.json', 'w', encoding='utf-8') as json_file:
    json.dump(results, json_file, indent=4, ensure_ascii=False)

In [25]:
connect_to_db('books.db')
create_three_table()
insert_data_into_tables()
execute_query(query1)
execute_query(query2)
execute_query(query3)
execute_query(query4)
execute_query(query5)
execute_query(query6)
execute_query(query7)